In [1]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to downscale daily ensemble P and T to hourly

import numpy as np
import os
import pandas as pd
import xarray as xr
import netCDF4 as nc
import datetime

def read_ens(out_forc_name_base, ens_num):
    for i in range(ens_num):
        ens_file = os.path.join(out_forc_name_base + '.' + str('%03d' % (i+1)) +'.nc')
        
        f=xr.open_dataset(ens_file)
        pcp = f.variables['pcp'][:]
        t_mean = f.variables['t_mean'][:]
        t_range = f.variables['t_range'][:]

        if i == 0:
            time = pd.DatetimeIndex(f['time'][:].dt.floor('D').to_pandas())
                 
            pcp_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))# create ens array 
            t_mean_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))
            t_range_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))

        pcp_ens[:,:,:,i] = pcp
        t_mean_ens[:,:,:,i] = t_mean
        t_range_ens[:,:,:,i] = t_range
       
    return time, pcp_ens, t_mean_ens, t_range_ens

#======================================================================================================
# main script
root_dir='/glade/u/home/hongli/scratch/2019_10_01gssha/ens_forc_wrf2' # cheyenne
# root_dir='/home/hongli/work/russian/ens_forc' #hydro-c1

wrf_hour_file = os.path.join(root_dir,'scripts/step1_asc_to_nc/WestWRF_2017120200_2018040723.nc')
wrf_day_file = os.path.join(root_dir,'scripts/step1_asc_to_nc/WestWRF_daily_20171202_20180407.nc')
result_dir = os.path.join(root_dir,'test_uniform_cv')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
ens_num = 100

output_dir=os.path.join(root_dir, 'scripts/step11_downscale_daily2hr')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_filename_base = 'ens_forc.'
nc_tpl=os.path.join(root_dir,'scripts/step1_asc_to_nc/WestWRF_2017120200_2018040723.nc')
    
#======================================================================================================
# read historical hourly wrf data
print('Read hourly WRF data')
f = xr.open_dataset(wrf_hour_file)
pcp_hour = f['Prcp'].values[:] # (time, lat, lon). unit: mm/hr
temp_hour = f['Temp'].values[:] # (time, lat, lon). unit: Fahrenheit
(nt_hour,ny,nx)=np.shape(pcp_hour)

#======================================================================================================
# read historical daily wrf data
print('Read daily WRF data')
f = xr.open_dataset(wrf_day_file)
time = pd.DatetimeIndex(f['time'][:].dt.floor('D').to_pandas())
pcp_day = f['prcp'].values[:] # (time, lat, lon). unit: mm/day
tmin_day = np.multiply(f['tmin'].values[:], 9.0/5.0) + 32 # unit: F. (0°C × 9/5) + 32 = 32°F.
tmax_day = np.multiply(f['tmax'].values[:], 9.0/5.0) + 32
(nt_day,ny,nx)=np.shape(pcp_day)

#======================================================================================================
# calcualte percent for disaggregation
print('Calculate portion')
# repeat daily total P and Trange into shape (nt_hour,ny,nx)
pcp_day_repeat = np.empty_like(pcp_hour)
tmin_day_repeat = np.empty_like(temp_hour)
tmax_day_repeat = np.empty_like(temp_hour)
for d in range(nt_day):
    hr_num = 24
    for i in range(ny):
        for j in range(nx):
            pcp_day_repeat[hr_num*(d):hr_num*(d+1),i,j]=pcp_day[d,i,j] 
            tmin_day_repeat[hr_num*(d):hr_num*(d+1),i,j]=tmin_day[d,i,j] 
            tmax_day_repeat[hr_num*(d):hr_num*(d+1),i,j]=tmax_day[d,i,j] 

# calcualte portion
# prcp_prct = p_hour/p_day. The sum of prcp_prct on the same day is one.
# temp_prct = (temp-tmin)/(tmax-tmin). The min and max temp_prct on the same day are zero and one, respectively.
prcp_prct = np.divide(pcp_hour,pcp_day_repeat,
                      out=np.ones_like(pcp_day_repeat)*(1/24.0),where=pcp_day_repeat!=0) #p/p_total
temp_prct = np.divide(temp_hour-tmin_day_repeat,tmax_day_repeat-tmin_day_repeat,
                     out=np.ones_like(tmin_day_repeat)*(1/24.0),where=(tmax_day_repeat-tmin_day_repeat)!=0) #t/(tmax-tmin)

#======================================================================================================
print('Downscale')
# loop through all uniform tests
for test_folder in test_folders[0:1]:
    
    print(test_folder)
    test_dir = os.path.join(result_dir, test_folder)
    if not os.path.exists(os.path.join(output_dir,test_folder)):
        os.makedirs(os.path.join(output_dir,test_folder))

    for i in range(ens_num):
        
        # read ensemble
        ens_file = 'ens_forc.'+ str('%03d' % (i+1)) +'.nc'
        f=xr.open_dataset(os.path.join(test_dir, 'outputs', ens_file))
        time = pd.DatetimeIndex(f['time'][:].dt.floor('D').to_pandas())
        pcp = f.variables['pcp'][:]
        tmean = f.variables['t_mean'][:] # C 
        trange = f.variables['t_range'][:] 
        
        tmin = tmean - 0.5*trange
        tmax = tmean + 0.5*trange        
        tmin = np.multiply(tmin, 9.0/5.0) + 32 # C to F.
        tmax = np.multiply(tmax, 9.0/5.0) + 32
    
        # extend daily ensemble to hourly size
        pcp_repeat = np.empty_like(pcp_hour)
        tmin_repeat = np.empty_like(temp_hour)
        tmax_repeat = np.empty_like(temp_hour)
        hr_num = 24
        for d in range(len(time)):
            pcp_repeat[hr_num*d:hr_num*(d+1),:,:]=pcp[d,:,:] 
            tmin_repeat[hr_num*d:hr_num*(d+1),:,:]=tmin[d,:,:] 
            tmax_repeat[hr_num*d:hr_num*(d+1),:,:]=tmax[d,:,:]

        # calculate hourly value for an ensemble member. (nt_hour,ny,nx).
        pcp_hour_ens = np.multiply(prcp_prct, pcp_repeat) 
        temp_hour_ens = np.multiply(temp_prct, (tmax_repeat-tmin_repeat))+tmin_repeat
        
        # write to a netcdf file        
        with nc.Dataset(nc_tpl) as src:
            with nc.Dataset(os.path.join(output_dir,test_folder, ens_file), "w") as dst:
                
                # copy dimensions
                for name, dimension in src.dimensions.items():
                     dst.createDimension(
                        name, (len(dimension) if not dimension.isunlimited() else None))
        
                # copy variable attributes all at once via dictionary (for the included variables)
                include = ['lat', 'lon', 'time', 'Temp','Prcp']
                for name, variable in src.variables.items():
                    if name in include:
                        x = dst.createVariable(name, variable.datatype, variable.dimensions)               
                        dst[name].setncatts(src[name].__dict__)
                        if name!='Temp' or name!='Prcp':
                            dst[name][:]=src[name][:]                
                
                dst.variables['Prcp'][:] = pcp_hour_ens
                dst.variables['Temp'][:] = temp_hour_ens    

        del pcp,tmean,trange,pcp_repeat,tmin_repeat,tmax_repeat,pcp_hour_ens,temp_hour_ens 

print('Done')


Read hourly WRF data
Read daily WRF data
Calculate portion
Downscale
046grids
Done


In [3]:
i=10
j=10
d=1
hr_num=24
pcp_day_repeat[hr_num*(d):hr_num*(d+1),i,j],pcp_day[d,i,j]

(array([0.68468742, 0.68468742, 0.68468742, 0.68468742, 0.68468742,
        0.68468742, 0.68468742, 0.68468742, 0.68468742, 0.68468742,
        0.68468742, 0.68468742, 0.68468742, 0.68468742, 0.68468742,
        0.68468742, 0.68468742, 0.68468742, 0.68468742, 0.68468742,
        0.68468742, 0.68468742, 0.68468742, 0.68468742]),
 0.6846874208356961)

In [4]:
pcp_hour[hr_num*(d):hr_num*(d+1),i,j],prcp_prct[hr_num*(d):hr_num*(d+1),i,j]

(array([1.19631960e-04, 1.41492800e-04, 4.55140730e-02, 9.08866500e-02,
        1.36259240e-01, 1.21451326e-01, 1.06643416e-01, 9.18355100e-02,
        6.12237380e-02, 3.06119640e-02, 1.89537840e-07, 1.26358570e-07,
        6.31792860e-08, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 array([1.74724928e-04, 2.06653132e-04, 6.64742357e-02, 1.32741814e-01,
        1.99009411e-01, 1.77382149e-01, 1.55754893e-01, 1.34127643e-01,
        8.94185232e-02, 4.47094003e-02, 2.76823897e-07, 1.84549279e-07,
        9.22746411e-08, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))

In [5]:
pcp_hour[hr_num*(d):hr_num*(d+1),i,j]/pcp_day_repeat[hr_num*(d):hr_num*(d+1),i,j]

array([1.74724928e-04, 2.06653132e-04, 6.64742357e-02, 1.32741814e-01,
       1.99009411e-01, 1.77382149e-01, 1.55754893e-01, 1.34127643e-01,
       8.94185232e-02, 4.47094003e-02, 2.76823897e-07, 1.84549279e-07,
       9.22746411e-08, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [32]:
pcp_hour[0:8,i,j],prcp_prct[0:8,i,j]

(array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667]))

In [30]:
pcp_hour[-3:,i,j]/pcp_day_repeat[-3:,i,j]

array([0.17851137, 0.33333331, 0.48815532])

In [8]:
a=np.where(pcp_day_repeat==0,1/24.0,np.divide(pcp_hour,pcp_day_repeat))

<ipython-input-8-806a62e80e7e>:1: RuntimeWarning: divide by zero encountered in true_divide
  a=np.where(pcp_day_repeat==0,1/24.0,np.divide(pcp_hour,pcp_day_repeat))
<ipython-input-8-806a62e80e7e>:1: RuntimeWarning: overflow encountered in true_divide
  a=np.where(pcp_day_repeat==0,1/24.0,np.divide(pcp_hour,pcp_day_repeat))
<ipython-input-8-806a62e80e7e>:1: RuntimeWarning: invalid value encountered in true_divide
  a=np.where(pcp_day_repeat==0,1/24.0,np.divide(pcp_hour,pcp_day_repeat))


In [9]:
a[8:8+24,10,10]

array([4.16666667e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.16666667e-02, 4.16666667e-02, 4.16666667e-02,
       0.00000000e+00, 4.16666667e-02, 4.16666667e-02, 0.00000000e+00,
       4.16666667e-02, 4.16666667e-02, 4.16666667e-02, 4.16666667e-02,
       8.64554878e-05, 1.11102582e-01, 2.22118707e-01, 3.33134832e-01,
       2.22123973e-01, 1.11113100e-01, 1.02227202e-04, 1.31666878e-04])